# Práctico 4

## Transform Pattern

**"Transform pattern"** en el contexto de Machine Learning se refiere a la técnica de manipular y cambiar los datos de entrada antes de que sean utilizados por un modelo. 

Estos cambios pueden ayudar a mejorar el rendimiento del modelo y a hacer que el modelo sea más robusto ante variaciones en los datos de entrada.

Las transformaciones se aplican típicamente durante la etapa de pre_procesamiento de los datos y pueden implicar muchas técnicas diferentes, dependiendo del tipo de datos y del problema que se está tratando de resolver.

In [ ]:
#!pip install tensorflow

In [ ]:
%%time 

import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt

# Cargar y normalizar el conjunto de datos 
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Convertir las etiquetas en one-hot
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

data_preprocessing = Sequential([
    preprocessing.Resizing(32, 32),  # Redimensionar a 32x32
    preprocessing.Rescaling(1./255)  # Normalización adicional después del redimensionamiento
])

# Define tu modelo
model = tf.keras.models.Sequential([
    data_preprocessing,

    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 1)),  # Cambio de 3 a 1 canal de entrada
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Compila y entrena el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(x_train, y_train, batch_size=256, epochs=5, validation_data=(x_test, y_test))

# Guarda el modelo
model.save('transform_pattern_conv.h5')

En este caso, las transformaciones de datos **(Resizing y Rescaling)** se definen como capas Keras y se agregan al inicio de tu modelo. 

Esto significa que estas transformaciones se aplicarán automáticamente a las imágenes a medida que pasen por el modelo, ya sea durante el entrenamiento o durante la inferencia. 

Además, como las capas de preprocesamiento son parte del modelo, se guardarán junto con el modelo cuando lo guardes con model.save(). 

## Experiment tracking
### wandb: https://wandb.ai/site


Primero, vamos a agregar experiment tracking utilizando wandb (Weights & Biases). Esto nos va a permitir monitorear los experimentos en tiempo real, guardar nuestros modelos , resultados, y podremos compartir experimentos con otros.

[Wandb collab full explained notebook ](https://colab.research.google.com/github/wandb/examples/blob/master/colabs/intro/Intro_to_Weights_%26_Biases.ipynb#scrollTo=jufPgkgqz2eF)

### 👟 Run an experiment 

1.  **Start a new run** and pass in hyperparameters to track

2.  **Log metrics** from training or evaluation

3.  **Visualize results** in the dashboard

4. **Generate alerts** in the dashboard 

In [ ]:
#!pip install wandb

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import wandb
from wandb.keras import WandbCallback

wandb.login()

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt
import random

from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint

# Launch 2 experiments, trying different dropout rates
for run in range(2):
    
    # Start a run, tracking hyperparameters
    wandb.init(
        project="ml-en-produccion",
        config={
        
            "activation_1": "relu",
            "dropout": random.uniform(0.01, 0.80),
            "optimizer": "adam",
            "loss": "categorical_crossentropy",
            "metric": "accuracy",
            "epoch": 3,
            "batch_size": 512,
        },
    )
    config = wandb.config
    
    # Cargar y normalizar el conjunto de datos 
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()

    # Convertir las etiquetas en one-hot
    y_train = tf.keras.utils.to_categorical(y_train, 10)
    y_test = tf.keras.utils.to_categorical(y_test, 10)

    # Define el preprocesamiento de la imagen

    data_preprocessing = Sequential([
        preprocessing.Resizing(32, 32),  # Redimensionar a 32x32
        preprocessing.Rescaling(1./255)  # Normalización adicional después del redimensionamiento
    ])

    # Define tu modelo
    model = tf.keras.models.Sequential([
        data_preprocessing,  

        tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Dropout(0.3),

        tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Dropout(config.dropout),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(config.dropout),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    # Compila y entrena el modelo
    model.compile(optimizer=config.optimizer, loss=config.loss,  metrics=[config.metric])

    # Add WandbMetricsLogger to log metrics and WandbModelCheckpoint to log model checkpoints

    wandb_callbacks = [
        WandbMetricsLogger(),
        WandbModelCheckpoint(filepath="my_model_{epoch:02d}"),
    ]

    history = model.fit(x_train, y_train, batch_size=config.batch_size, epochs=config.epoch, validation_data=(x_test, y_test), callbacks=wandb_callbacks)
    
    wandb.finish()
    
    #save model 
    model.save('model.h5')


##  W&B Alerts

**[W&B Alerts](https://docs.wandb.ai/guides/track/alert)** allows you to send alerts, triggered from your Python code, to your Slack or email. There are 2 steps to follow the first time you'd like to send a Slack or email alert, triggered from your code:

1) Turn on Alerts in your W&B [User Settings](https://wandb.ai/settings)

2) Add `wandb.alert()` to your code:

```python
wandb.alert(
    title="Low accuracy", 
    text=f"Accuracy is below the acceptable threshold"
)
```

In [ ]:
import random 

# Start a wandb run
wandb.init(project="alerts-intro")

# Simulating a model training loop
acc_threshold = 0.3
for training_step in range(1000):

    # Generate a random number for accuracy
    accuracy = round(random.random() + random.random(), 3)
    print(f"Accuracy is: {accuracy}, {acc_threshold}")

    # 🐝 Log accuracy to wandb
    wandb.log({"Accuracy": accuracy})

    # 🔔 If the accuracy is below the threshold, fire a W&B Alert and stop the run
    if accuracy <= acc_threshold:
        # 🐝 Send the wandb Alert
        wandb.alert(
            title="Low Accuracy",
            text=f"Accuracy {accuracy} at step {training_step} is below the acceptable theshold, {acc_threshold}",
        )
        print("Alert triggered")
        break

# Mark the run as finished (useful in Jupyter notebooks)
wandb.finish()

## H Tuning - wandb

In [ ]:
sweep_config = {
    'method': 'random',  # el método de búsqueda de hiperparámetros
    'metric': {
      'name': 'accuracy',
      'goal': 'maximize'  
    },
    'parameters': {
        'learning_rate': {
            'values': [0.1, 0.01, 0.001, 0.0001]
        },
        'batch_size': {
            'values': [64, 128, 256]
        },
    }
}

sweep_id = wandb.sweep(sweep_config, project="Htuning")

def train():
    run = wandb.init()
    config = run.config

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=config.learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(x_train, y_train, batch_size=config.batch_size, epochs=5, validation_data=(x_test, y_test), callbacks=[WandbCallback()])

wandb.agent(sweep_id, function=train)